# Unit testing: pytest
*Davide Gerosa (Milano-Bicocca)*

**Sources**: Michael Zingale at Stony Brook University: https://sbu-python-class.github.io

Testing is an integral part of the software development process.  We want to catch
mistakes early, before the go on to affect our results.

## Types of testing

There are a lot of different types of software testing that exist.
Most commonly, for scientific codes, we hear about:

* Unit testing : Tests that a single function does what it was designed to do

* Integration testing : Tests whether the individual pieces work together as intended.
  Sometimes done one piece at a time (iteratively)

* Regression testing : Checks whether changes have changed answers

* Verification & Validation (from the science perspective)

  * Verification: are we solving the equations correctly?

  * Validation: are we solving the correct equations?

## Automating testing

The best testing is automated.  Github provides a *continuous integration* service that can
be run on pull requests.  You write a short definition (a Github workflow) that tells Github
how to run your tests and then any time there is a change, the tests are run.

## Unit testing

* When to write tests?

  * Some people advocate writing a unit test for a specification
    before you write the functions they will test

    * This is called Test-driven development (TDD):
      https://en.wikipedia.org/wiki/Test-driven_development

  * This helps you understand the interface, return values,
    side-effects, etc. of what you intend to write

* Often we already have code, so we can start by writing tests to
  cover some core functionality

  * Add new tests when you encounter a bug, precisely to ensure that
    this bug doesn't arise again

* Tests should be short

  * You want to be able to run them frequently





# pytest

`pytest` is a unit testing framework for python code.

Basic elements:

* Discoverability: it will find the tests

* Automation

* Fixtures (setup and teardown)

## Installing

You can install `pytest` for a single user as:

```
pip install pytest
```

This should put `pytest` in your search path, likely in `~/.local/bin`.

If you want to generate coverage reports, you should also install `pytest-cov`:

```
pip install pytest-cov
```

## Test discovery

Adhering to these naming conventions will ensure that your tests are automatically found:

* File names should start or end with “test”:

  * `test_example.py`
  * `example_test.py`

* For tests in a class, the class name should begin with `Test`

  * e.g., `TestExample`
  * There should be no `__init__()`

* Test method / function names should start with `test_`

  * e.g., `test_example()`

## Assertions

Tests use assertions (via python’s `assert` statement) to check behavior at runtime

* https://docs.python.org/3/reference/simple_stmts.html#assert 

* Basic usage: `assert expression`

  * Raises `AssertionError` if expression is not true

  * e.g., `assert 1 == 0` will fail with an exception

## Simple pytest example

Create a file named `test_simple.py` with the following content:

```python
def multiply(a, b):
    return a*b

def test_multiply():
    assert multiply(4, 6) == 24

def test_multiply2():
    assert multiply(5, 6) == 2
```

then we can run the tests as:

```
pytest -v
```

and we get the output:

```
============================= test session starts ==============================
platform linux -- Python 3.11.3, pytest-7.2.2, pluggy-1.0.0 -- /usr/bin/python3
cachedir: .pytest_cache
rootdir: /home/zingale/temp/pytest
plugins: anyio-3.6.2
collected 2 items                                                              

test_simple.py::test_multiply PASSED                                     [ 50%]
test_simple.py::test_multiply2 FAILED                                    [100%]

=================================== FAILURES ===================================
________________________________ test_multiply2 ________________________________

    def test_multiply2():
>       assert multiply(5, 6) == 2
E       assert 30 == 2
E        +  where 30 = multiply(5, 6)

test_simple.py:8: AssertionError
=========================== short test summary info ============================
FAILED test_simple.py::test_multiply2 - assert 30 == 2
========================= 1 failed, 1 passed in 0.04s ==========================
```

this is telling us that one of our tests has failed.


# More pytest

Unit tests sometimes require some setup to be done before the test is run.  Fixtures
provide this capability.

pytest provides `setup` and `teardown` functions/methods for tests --
see https://docs.pytest.org/en/6.2.x/fixture.html for more details

Note:  By default, pytest will capture stdout and only show it on failures.  If you want
to always show stdout, add the `-s` flag.


## Example class

It is common to use a class to organize a set of related unit tests.  This is
not a full-fledged class -- it simply helps to organize data.  In particular,
there is no constructor, `__init__()`.  See https://stackoverflow.com/questions/21430900/py-test-skips-test-class-if-constructor-is-defined

We'll look at an example with a NumPy array

* We always want the array to exist for our tests, so we'll use
  fixtures (in particular `setup_method()`) to create the array

* Using a class means that we can access the array created in setup from our class.

* We'll use numpy's own assertion functions: https://numpy.org/doc/stable/reference/routines.testing.html


Here's an example:

```python
# a test class is useful to hold data that we might want setup
# for every test.

import numpy as np
from numpy.testing import assert_array_equal

class TestClassExample:

    @classmethod
    def setup_class(cls):
        """ this is run once for each class, before any tests """
        pass

    @classmethod
    def teardown_class(cls):
        """ this is run once for each class, after all tests """
        pass

    def setup_method(self):
        """ this is run before each of the test methods """
        self.a = np.arange(24).reshape(6, 4)

    def teardown_method(self):
        """ this is run after each of the test methods """
        pass

    def test_max(self):
        assert self.a.max() == 23

    def test_flat(self):
        assert_array_equal(self.a.flat, np.arange(24))
```


Here we see the [`@classmethod` decorator](https://docs.python.org/3/library/functions.html#classmethod).
This means that the function receives the class itself as the first argument rather then an instance,
e.g., `self`.


Put this into a file called `test_class.py` and then we can run as:

```
pytest -v
```



## Modern workflow

Modern code tested is collaborative and automated. Say you found a bug in numpy and want to fix it. You can only pull request to their git repo only if your patch passes a whole suite of tests.

That's for big things, but I argue having some basic unit tests is important for every code, even your little PhD project (and well... if you end up having a pull request merged into numpy, that should go straight into your CV!!!)


## Other types of tests

Unit tests are only one form of testing - they test a function in
isolation of others.  Sometimes we need to test everything working together.
For scientific codes, regression testing is often used.  The basic workflow
is:

* Start with the project working in a way you are happy with

* Store the output of one (or more) runs as a _benchmark_.

* Each time you make changes, run the code and compare the new output
  to the stored benchmark.

  * If there are no differences, then your changes are likely good
    (but there is always the case of some feature not being tested).

  * If there are differences, then either you introduced a bug, in which
    case you should fix it, or you fixed a bug, in which case you should
    update the benchmarks.


# Excercises

## Q1: My own test
Pick a piece of python code that you like (your own PhD project, or take one of the exercises from this class). Implement a unit test and a regression test. 

Optional: write a github action that does it for you. 


## Q2: How do professionals do it?
Pick a big git repo, say [scipy](https://github.com/scipy/scipy) or [numpy](https://github.com/numpy/numpy),  and have a look at their development workflow, including their testing strategies.
